# Segmenting and Clustering Neighborhoods in Toronto

## The procedure will be as follows
1 - Scrapping the data from Wikipedia  
2 - Transforming the data into the appropriate dataframe


### 1 - Scrapping data from Wikipedia

1.1 - Importing libraries

In [1]:
import bs4 as bs
import urllib.request
import numpy as np
import pandas as pd 
import folium
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

1.2 - setting the url

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

1.3 - getting data

In [3]:
def scrape_url(cname,cols):
    page  = urllib.request.urlopen(url).read()
    soup  = bs.BeautifulSoup(page,'lxml')
    table = soup.find("table",class_=cname)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    # Store data to this temporary dataframe
    raw_df = pd.DataFrame(data,columns=header)
    return raw_df

In [4]:
raw_data = scrape_url("wikitable",3)

In [5]:
raw_data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 2 - Transforming the data into the appropriate dataframe

2.1 removal of codes with unassigned Borough

In [6]:
postal_codes_df=raw_data[~raw_data['Borough'].isin(['Not assigned'])]

2.2 assign borough to unassigned Neighbourhoods

In [7]:
postal_codes_df.loc[postal_codes_df['Neighbourhood'] == 'Not assigned','Neighbourhood'] = postal_codes_df[postal_codes_df['Neighbourhood'] == 'Not assigned']['Borough']
# THIS CALL THROWS A WARNING, PLEASE IGNORE IT

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


2.3 group neighbourhoods with the same postal code

In [8]:
postal_codes_df = postal_codes_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [9]:
postal_codes_df.shape

(103, 3)

# PART II

the geospatial csv file is used

In [10]:
!wget -q -O 'toronto_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


prepare to load the data from the csv file

In [11]:
filename = 'toronto_data.csv'
headers = ['Postal Code', 'Latitude', 'Longitude']

load and preview the data

In [12]:
df = pd.read_csv(filename)
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


add the coordinates to dataframe with the postal code

In [13]:
result = pd.merge(postal_codes_df, df, left_on='Postcode', right_on='Postal Code')

remove the extra column and preview the final output

In [14]:
result.drop('Postal Code', axis=1, inplace=True)
result.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# PART III

### Plot and fetch neighbourhood details

filter neighbourhoods with Toronto in their name

In [15]:
selected = result[result['Neighbourhood'].str.contains('Toronto')]
selected.shape

(7, 5)

get Toronto coordinates using geopy

In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


plot the selected neighbourhoods on the map

In [17]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for Latitude, Longitude, Borough, Neighbourhood in zip(selected['Latitude'], selected['Longitude'], selected['Borough'], selected['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### set foursquare creds

In [18]:
CLIENT_ID = 'L4TEIJMILCEPTFI34OPPLJYNRPLUFD4BZZMJUKA1Z0DD2TFH' # your Foursquare ID
CLIENT_SECRET = '5QVK35FPKSKTNVOGJTKXCGC4RDAQUNKZHVZZCKZ52ARK2C5K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L4TEIJMILCEPTFI34OPPLJYNRPLUFD4BZZMJUKA1Z0DD2TFH
CLIENT_SECRET:5QVK35FPKSKTNVOGJTKXCGC4RDAQUNKZHVZZCKZ52ARK2C5K


### we'll get the top 100 venues for each neighbourhood and then we'll cluster them based on frequency of visit

#### get the top 100 venues of each neighbourhood

set the url

In [19]:
search_query = ''
radius = 500
LIMIT = 100
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}& \
radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

 .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=L4TEIJMILCEPTFI34OPPLJYNRPLUFD4BZZMJUKA1Z0DD2TFH&client_secret=5QVK35FPKSKTNVOGJTKXCGC4RDAQUNKZHVZZCKZ52ARK2C5K&ll=43.653963,-79.387207&v=20180605& radius=&limit=500'

call the foursquare api and store the results

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c93b605351e3d4c7df49b0d'},
 'response': {'venues': [{'id': '4f04779a02d5cce0cfc06151',
    'name': 'Chinese Visa Application Service Center',
    'location': {'address': '393 University Ave, Suite 1501',
     'crossStreet': 'in University Centre',
     'lat': 43.65402839343005,
     'lng': -79.38736458967021,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65402839343005,
       'lng': -79.38736458967021}],
     'distance': 14,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['393 University Ave, Suite 1501 (in University Centre)',
      'Toronto ON',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d126941735',
      'name': 'Government Building',
      'pluralName': 'Government Buildings',
      'shortName': 'Government',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_',
       'suffix': '.png'},
      'primary': True}],
 

Let's borrow the **get_category_type** function from the Foursquare lab.

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

transfer the foursquare response to a dataframe

In [22]:
venues = results['response']['venues']
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.shape
# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# # filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# # clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Chinese Visa Application Service Center,Government Building,43.654028,-79.387365
1,Ontario Superior Court of Justice,Courthouse,43.653521,-79.387193
2,393 University Ave,Office,43.653752,-79.388150
3,TD Canada Trust,Bank,43.655062,-79.387840
4,Nobs',Food Truck,43.653930,-79.387204


let's also borrow the function to get the data of all neighborhoods from the foursquare lab

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

and use it to get the venues from our foursquare response

In [24]:
toronto_venues = getNearbyVenues(names=selected['Neighbourhood'],
                                   latitudes=selected['Latitude'],
                                   longitudes=selected['Longitude']
                                  )

CFB Toronto, Downsview East
East Toronto
North Toronto West
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Harbord, University of Toronto
Humber Bay Shores, Mimico South, New Toronto


check the results

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(277, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"CFB Toronto, Downsview East",43.737473,-79.464763,Toronto Downsview Airport (YZD),43.738883,-79.470111,Airport
1,"CFB Toronto, Downsview East",43.737473,-79.464763,Ttc Bus #120 - Plewes Rd,43.734898,-79.464221,Bus Stop
2,"CFB Toronto, Downsview East",43.737473,-79.464763,Ancaster Park,43.734706,-79.464777,Park
3,East Toronto,43.685347,-79.338106,The Path,43.683923,-79.335007,Park
4,East Toronto,43.685347,-79.338106,Sammon Convenience,43.686951,-79.335007,Convenience Store


group by neighborhood to get the count of venues in each

In [26]:
toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"CFB Toronto, Downsview East",3,3,3,3,3,3
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
East Toronto,3,3,3,3,3,3
"Harbord, University of Toronto",34,34,34,34,34,34
"Harbourfront East, Toronto Islands, Union Station",100,100,100,100,100,100
"Humber Bay Shores, Mimico South, New Toronto",14,14,14,14,14,14
North Toronto West,23,23,23,23,23,23


let's count the unique categories in our data set

In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 103 uniques categories.


convert the table to one hot encoding

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Stadium,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,"CFB Toronto, Downsview East",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"CFB Toronto, Downsview East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
toronto_onehot.shape

(277, 104)

let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Stadium,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,"CFB Toronto, Downsview East",0.333333,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000
1,"Design Exchange, Toronto Dominion Centre",0.000000,0.04,0.00,0.01,0.01,0.010000,0.00,0.020000,0.00,...,0.000000,0.01,0.01,0.010000,0.000000,0.01,0.01,0.000000,0.01,0.000000
2,East Toronto,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000
3,"Harbord, University of Toronto",0.000000,0.00,0.00,0.00,0.00,0.058824,0.00,0.058824,0.00,...,0.029412,0.00,0.00,0.029412,0.000000,0.00,0.00,0.029412,0.00,0.000000
4,"Harbourfront East, Toronto Islands, Union Station",0.000000,0.00,0.05,0.01,0.00,0.020000,0.01,0.020000,0.02,...,0.010000,0.01,0.00,0.010000,0.000000,0.01,0.01,0.000000,0.01,0.000000
5,"Humber Bay Shores, Mimico South, New Toronto",0.000000,0.00,0.00,0.00,0.00,0.071429,0.00,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000
6,North Toronto West,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,...,0.000000,0.00,0.00,0.000000,0.043478,0.00,0.00,0.000000,0.00,0.043478


In [48]:
toronto_grouped.shape

(7, 104)

print each neighborhood along with the top 5 most common venues

In [47]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CFB Toronto, Downsview East----
                 venue  freq
0              Airport  0.33
1             Bus Stop  0.33
2                 Park  0.33
3          Salad Place  0.00
4  Rental Car Location  0.00


----Design Exchange, Toronto Dominion Centre----
                 venue  freq
0          Coffee Shop  0.14
1                 Café  0.08
2                Hotel  0.07
3  American Restaurant  0.04
4   Seafood Restaurant  0.03


----East Toronto----
               venue  freq
0  Convenience Store  0.33
1        Coffee Shop  0.33
2               Park  0.33
3        Salad Place  0.00
4         Restaurant  0.00


----Harbord, University of Toronto----
                 venue  freq
0                 Café  0.15
1           Restaurant  0.06
2               Bakery  0.06
3                  Bar  0.06
4  Japanese Restaurant  0.06


----Harbourfront East, Toronto Islands, Union Station----
                venue  freq
0         Coffee Shop  0.13
1            Aquarium  0.05
2               Hotel

#### Let's put that into a *pandas* dataframe

let's borrow the function to sort the venues in descending order.

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

let's create the new dataframe and display the top 10 venues for each neighborhood.

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CFB Toronto, Downsview East",Airport,Bus Stop,Park,Comfort Food Restaurant,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
1,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,American Restaurant,Deli / Bodega,Italian Restaurant,Gastropub,Restaurant,Seafood Restaurant,Bar
2,East Toronto,Coffee Shop,Convenience Store,Park,Yoga Studio,Comfort Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store
3,"Harbord, University of Toronto",Café,Bar,Bookstore,Japanese Restaurant,Restaurant,Bakery,Chinese Restaurant,Poutine Place,Coffee Shop,College Arts Building
4,"Harbourfront East, Toronto Islands, Union Station",Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Brewery,Pizza Place,Restaurant,Fried Chicken Joint,Scenic Lookout


In [51]:
selected.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
30,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763
40,M4J,East York,East Toronto,43.685347,-79.338106
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576


Run *k*-means to cluster the neighborhood into 5 clusters.

In [36]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
toronto_grouped_clustering.head()

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 2, 0, 0, 4, 3], dtype=int32)

In [52]:
selected.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
30,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763
40,M4J,East York,East Toronto,43.685347,-79.338106
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
60,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = selected

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

Finally, let's visualize the resulting clusters

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Let's examine each cluster and study its nodes

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Brewery,Pizza Place,Restaurant,Fried Chicken Joint,Scenic Lookout
60,Downtown Toronto,0,Coffee Shop,Café,Hotel,American Restaurant,Deli / Bodega,Italian Restaurant,Gastropub,Restaurant,Seafood Restaurant,Bar
66,Downtown Toronto,0,Café,Bar,Bookstore,Japanese Restaurant,Restaurant,Bakery,Chinese Restaurant,Poutine Place,Coffee Shop,College Arts Building


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,North York,1,Airport,Bus Stop,Park,Comfort Food Restaurant,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,East York,2,Coffee Shop,Convenience Store,Park,Yoga Studio,Comfort Food Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Central Toronto,3,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Shoe Store,Gift Shop,Ice Cream Shop,Italian Restaurant,Fast Food Restaurant,Mexican Restaurant


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,Etobicoke,4,Pizza Place,Fried Chicken Joint,Seafood Restaurant,Fast Food Restaurant,Flower Shop,Mexican Restaurant,Sandwich Place,Liquor Store,Café,Bakery
